<a href="https://colab.research.google.com/github/hudada369/fy2020-repo-config/blob/master/Pytorch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 模型的保存与加载
2. 模型的 finetune
3. GPU 使用
4. Pytorch 的常见报错

# **模型的保存与加载**
1. 序列化和反序列化（序列化就是说内存中的某一个对象保存到硬盘当中，以二进制序列的形式存储下来，而反序列化，就是将硬盘中存储的二进制的数，反序列化到内存当中）
  1. 序列化，torch.save(obj,f):obj表示对象，是保存的数据，可以是**模型，张量，dict等**，f表示输出的路径
  2. 反序列化：torch.load(f,map_location): f表示文件的路径，map_location 指定存档的位置，cpu或者GPU

2. 模型保存加载，保存module 或者是 模型参数
  1. torch.save(net,path),torch.load(fpath)
  2. orch.save（net.state_dict(),path）,net.load_state_dict(torch.load(path))
  3. 第一种保存整个模型架构，第二种是保留模型的可学习参数，等建立新的网络结构，然后放上参数

In [1]:
import torch
import torch.nn as nn
class LeNet2 (nn.Module):
    def __init__(self, classes):
        super (LeNet2, self).__init__()
        self.features = nn.Sequential (
            nn.Conv2d (3, 6, 5),
            nn.ReLU (),
            nn.MaxPool2d (2, 2),
            nn.Conv2d (6, 16, 5),
            nn.ReLU (),
            nn.MaxPool2d (2, 2)
        )
        self.classifier = nn.Sequential (
            nn.Linear (16*5*5, 120),
            nn.ReLU (),
            nn.Linear (120, 84),
            nn.ReLU (),
            nn.Linear (84, classes)
        )

    def forward (self, x):
        x = self.features (x)
        x = x.view (x.size ()[0], -1)
        x = self.classifier (x)
        return x

    def initialize (self):
        for p in self.parameters ():
            p.data.fill_(20191104)
     
## 建立一个网络
net = LeNet2 (classes=2019)

# "训练"
print ("训练前:", net.features [0].weight [0, ...])
net.initialize ()
print ("训练后:", net.features [0].weight [0, ...])

训练前: tensor([[[-0.0675, -0.0774,  0.0508,  0.0326,  0.0852],
         [ 0.0622,  0.0228, -0.1002, -0.0137, -0.0450],
         [-0.0127,  0.0416, -0.1017,  0.0559, -0.0743],
         [-0.0892,  0.0717,  0.0785, -0.0098, -0.0476],
         [-0.1080, -0.0502,  0.0157, -0.0495, -0.0076]],

        [[-0.0033,  0.0627, -0.0803,  0.0257,  0.0920],
         [-0.0760, -0.0762, -0.0724, -0.1119, -0.0410],
         [-0.0515,  0.0887, -0.0882,  0.0803,  0.0080],
         [ 0.0545,  0.1061, -0.0889,  0.0242, -0.0145],
         [ 0.0247, -0.0691,  0.0530, -0.1004, -0.0708]],

        [[ 0.0727,  0.0885,  0.0708,  0.0607, -0.0206],
         [-0.0686, -0.0543, -0.0313,  0.0009,  0.0181],
         [ 0.0042,  0.0329,  0.0418, -0.0284, -0.0334],
         [ 0.0086,  0.0800, -0.0451,  0.0923, -0.0376],
         [-0.0859,  0.0876, -0.0347,  0.0289, -0.0877]]],
       grad_fn=<SelectBackward0>)
训练后: tensor([[[20191104., 20191104., 20191104., 20191104., 20191104.],
         [20191104., 20191104., 20191104., 2

1. 通过torch.save（net）保存整个模型，pkl后缀的文件，加载直接torch.load(目标地址)，也可以直接通过print打印整个模型的结构
2. 保存模型参数就是net.state)dict(), 也是pkl后缀的文件，
加载的方式就是，先创建同样结构的网络，然后new_net.load_state_dict(torch.load(path))

# **模型断点续训练**
1. 解释：设置断点的目的，当发生情况下，再启动。可以从设置的断点继续运行
2. 问题：断点保存什么参数？
  1. 模型里面的可学习参数
  2.  优化器里的一些缓存
  3. 所以checkpoint需要保存：模型的数据，优化器的数据，还有迭代到了第几次
  4. checkpoint = {
    "model_state_dict": net_state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "epoch":epoch
  }
3. 如何设置？
  1. 在epoch中进行断点保存：
  2. 内容，首先是checkpoint，然后是路径，最后是torch.save(checkpoint,path)
4. 如何从断点开始：
  1. 首先导入断点保存的参数，是一个字典，对字典的值取出来即可
  2. net.load_state_dict(checkpoint["model_state_dict"])
  3. optimizer.load_state_dict(.......)
  4. start_spoch = chexkpoint["epoch"]
  5. 如果我们设置了学习率更新，因为在迭代的时候设置了 schedule.step(),在函数参数中有一个last_epoch ，将last_epoch = start_spoch,将学习率定在这个位置

# **模型的finetune**
1. 迁移学习，机器学习分支， 研究**源域的知识如何应用到目标域**，将源任务中学习到的知识运用到目标任务当中，用来提升目标任务里模型的性能
2. 思路：当我们的任务数据比较少，没法训练一个模型的时候，采用迁移学习的思路，将**类似任务**的模型迁移，在新任务上面，只需要微调一些参数就能比较好的应用于新的任务（cv和nlp不同）
3. 模型微调的步骤：
  1. 获取预训练模型参数（源任务当中学习到的知识）
  2. 加载模型（load_state_dict）将学习到的知识放到新的模型
  3. 修改输出层， 以适应新的任务
4. 模型微调的训练方法：
  1. 固定预训练的参数 (requires_grad=False; lr=0)
  2. Features Extractor 较小学习率 (params_group)
5. 模型微调的实现： 
  1. 固定预训练的参数
    1. 构架新模型
    2. 将训练好的参数导入到模型里面
    3. for param in new_net.parameters(): param.requires_grad=False冻结前面的层，不让训练，
    4. 替换fc层，在其他预训练任务里面也可以进行添加，例如 num_ftrs = new_net.fc.in_features, new_net.fc = nn.linear(num_ftrs,new_classes)
    5. new_net.fit_to(device)
  2. 不冻结前面的层，而是修改前面的参数学习率，因为优化器里面有参数组的概念，可以将网络的前面和后面分为不同的参数组，使用不同的学习率进行训练，当前面的学习率为0时，就和冻结时一样的效果了
    1. fc_param_id = list(map(id,net.fc_parameters()))# 返回的时候resnet后面全连接的内存地址
    2. base_params = filter(lambda p: id(p) not in fc_params_id,new_net.parameters()) resnet前面的卷积层
    3. optimizer = optim.SGD([
      {'params':base_params,"lr":LR*0},
      {"params":net.fc_parameters(),'lr':LR}
    ],momentum=0.9)  # 优化器的参数组，全面的卷积层学习率比较小，不想训练可以为0


# **`4. GPU的使用`**
1. 数据迁移至gpu， tensor 和 module
  1. CPU -> GPU：data.to ("cpu")
GPU -> CPU: data.to ("cuda")
  2. tensor.to (*args,kwargs)
  3. module.to (*args,kwargs) 
  4. 张量不执行 inplace， 所以上面看到需要等号重新赋值，而模型执行 inplace， 所以不用等号重新赋值
  5. 使用：
    1. device = torch.device("cuda:0" if torch.cuda.is_available() else"cpu")
    2. 执行上面的data.to
  6. 如果模型在 GPU 上， 那么数据也必须在 GPU 上才能正常运行。也就是说数据和模型必须在相同的设备上

2. torch.cuda常用方法：
  1. torch.cuda.device_count (): 计算当前可见可用的 GPU 数
  2. torch.cuda.get_device_name (): 获取 GPU 名称
  3. torch.cuda.manual_seed (): 为当前 GPU 设置随机种子
  4. torch.cuda.manual_seed_all (): 为所有可见可用 GPU 设置随机种子
  5. torch.cuda.set_device (): 设置主 GPU（默认 GPU）为哪一个物理 GPU（不推荐） 推荐的方式是设置系统的环境变量：os.environ.setdefault ("CUDA_VISIBLE_DEVICES","2,3")  通过这个方法合理的分配 GPU，使得多个人使用的时候不冲突。但是这里要注意一下， 这里的 2,3 指的是物理 GPU 的 2,3。但是在逻辑 GPU 上， 这里表示的 0,1。
  主GPU的概念在并行运算中有用

3. 多GPU并行运算：有一个主GPU，拿到数据之后进行分发，然后对其余GPU计算的结果进行收回，分发 -> 并行计算 -> 收回结果整合。
  1. 多GPU并行运算的机制实现：torch.nn.DataParallel: 包装模型，实现分发并行机制
    1. module: 需要包装分发的模型
    2. device_ids: 可分发的 gpu, 默认分发到所有的可见可用GPU， 通常这个参数不管它，而是在环境变量中管这个。
    3. output_device: 结果输出设备， 通常是输出到主 GPU
    4. 实现：在运行的时候，将模型包装进去即可 net = torch.nn.DataParallel(net)

4. GPU模型加载常见的问题：
  1. Runtimeerror：在cpu上面运行GPU保存的模型，只需要torch.load (path_state_dict, map_location="cpu")
  2. eRROR IN LOADING STATE_DICT，多GPU运行保存的模型，在普通模型上面使用这些参数的错误，原因，在多gpu运算对进行并行的包装，在参数前面会加上module.，直接加入到普通模型，会出现找不到参数的错误，需要重新设置OrderedDict保存这些参数，名字去掉上面的多余即可


In [ ]:

x = torch.ones ((3,3))
x = x.to (torch.float64)    # 转换数据类型


linear = nn.Linear (2,2)
linear.to (torch.double)  # 这样模型里面的可学习参数的数据类型变成 float64

gpu1 = torch.device ("cuda")
linear.to (gpu1)    # 把模型从 CPU 迁移到 GPU

In [ ]:
#下面这个代码是多 GPU 的时候，查看每一块 GPU 的缓存，并且排序作为逻辑 GPU 使用，
# 排在最前面的一般设置为我们的主 GPU：


def get_gpu_memory ():
        import platform
        if 'Windows' != platform.system ():
            import os
            os.system ('nvidia-smi -q -d Memory | grep -A4 GPU | grep Free > tmp.txt')
            memory_gpu = [int (x.split ()[2]) for x in open ('tmp.txt', 'r').readlines ()]
            os.system ('rm tmp.txt')
        else:
            memory_gpu = False
            print ("显存计算功能暂不支持 windows 操作系统")
        return memory_gpu


    gpu_memory = get_gpu_memory ()
    if not gpu_memory:
        print ("\ngpu free memory: {}".format (gpu_memory))
        gpu_list = np.argsort (gpu_memory)[::-1]

        gpu_list_str = ','.join (map (str, gpu_list))
        os.environ.setdefault ("CUDA_VISIBLE_DEVICES", gpu_list_str)
        device = torch.device ("cuda" if torch.cuda.is_available () else "cpu")

In [ ]:
# 解决多GPU运行保存的模型，在普通模型上面使用这些参数的错误
from collections import OrderedDict
    new_state_dict = OrderedDict ()
    for k, v in state_dict_load.items ():
        namekey = k [7:] if k.startswith ('module.') else k
        new_state_dict [namekey] = v
    print ("new_state_dict:\n {}".format (new_state_dict))

net.load_state_dict(new_state_dict)

# **Pytorch的常见报错**
1. https://shimo.im/docs/PvgHytYygPVGJ8Hv
2. 